In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/zzc2/

In [ ]:
!git clone https://github.com/facebookresearch/detr.git

Cloning into 'detr'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 265 (delta 0), reused 1 (delta 0), pack-reused 264
Receiving objects: 100% (265/265), 12.88 MiB | 20.19 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [ ]:
!conda install -c pytorch pytorch torchvision

/bin/bash: line 1: conda: command not found


In [ ]:
!conda install cython scipy
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

/bin/bash: line 1: conda: command not found
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-r5jq464d
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-r5jq464d
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=375614 sha256=039dd505381ee5cf79a299e7e2659cc7560d4612bdedd60b9cbd0573ea717d57
  Stored in directory: /tmp/pip-ephem-wheel-cache-edi74h8k/wheels/39/61/b4/480fbddb4d3d6bc34083e7397bc6f5d1381f79acc68e9f3511
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.7
    Uninstalling pycocotools-2.0.7:
      Successfully uninstalled pycocotools-2.0.7


In [ ]:
import json

def yolo_to_coco(yolo_labels_path, coco_output_path):
    coco_data = {"images": [], "annotations": [], "categories": [{"id": 1, "name": "object"}]}
    image_id = 0
    annotation_id = 0

    # Read YOLO label files
    with open(yolo_labels_path, "r") as f:
        for line in f:
            # Parse label line (assuming "<class_id> <x_center> <y_center> <width> <height>")
            class_id, x_center, y_center, width, height = map(float, line.split())

            # Convert YOLO bounding box to COCO format
            x_min = (x_center - width / 2)
            y_min = (y_center - height / 2)

            # Add image entry to COCO data
            coco_data["images"].append({"id": image_id, "file_name": "image.jpg", "width": 640, "height": 480})

            # Add annotation entry to COCO data
            coco_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": 1,  # Assuming only one category
                "bbox": [x_min, y_min, width, height],
                "area": width * height,
                "iscrowd": 0  # Assuming not a crowd annotation
            })

            # Increment IDs
            image_id += 1
            annotation_id += 1

    # Write COCO data to JSON file
    with open(coco_output_path, "w") as f:
        json.dump(coco_data, f)

# Example usage
yolo_labels_path = "content/drive/MyDrive/zzc2/dataset (1)/labels (1)/"
coco_output_path = "path/to/coco_annotations.json"
yolo_to_coco(yolo_labels_path, coco_output_path)


In [ ]:
import torch
import torchvision.transforms as T
from torch.utils.data import DataLoader
from transformers import DetrForObjectDetection
from transformers import DetrForObjectDetection, DetrForObjectDetectionConfig
from datasets import CocoDetection, CocoDetectionSplit

# Define your training parameters
batch_size = 2
num_epochs = 10
learning_rate = 1e-4

# Load the COCO dataset
train_dataset = CocoDetectionSplit(root="path/to/coco", annFile="path/to/train_annotation.json")
val_dataset = CocoDetectionSplit(root="path/to/coco", annFile="path/to/val_annotation.json")

# Define data transforms
transform = T.Compose([
    T.Resize((800, 800)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transforms to the datasets
train_dataset.transform = transform
val_dataset.transform = transform

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the DETR model and optimizer
config = DetrForObjectDetectionConfig.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", config=config)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the loss function (DETR uses a custom loss combining classification and bounding box regression)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for images, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(images, targets=targets)
        loss = loss_fn(outputs.logits, outputs.logits_bbox)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    # Print average loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader)}")

    # Validation loop (optional)
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for images, targets in val_loader:
            outputs = model(images, targets=targets)
            loss = loss_fn(outputs.logits, outputs.logits_bbox)
            val_loss += loss.item()

    # Print validation loss
    print(f"Validation Loss: {val_loss / len(val_loader)}")

# Save the trained model
model.save_pretrained("path/to/save/trained_model")


In [ ]:
!python -m torch.distributed.launch --nproc_per_node=8 --use_env main.py --coco_path /path/to/coco

In [ ]:
!python main.py --batch_size 2 --no_aux_loss --eval --resume https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth --coco_path /path/to/coco